In [4]:
from collections import Counter 

import pandas as pd
import numpy as np
import re

In [10]:
data_path = "data/"
df = pd.read_csv(data_path + "processed_train.csv")

In [11]:
# df = df[:200]
df

,raw_address,POI/street
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,setu siung 119 rt 5 1 13880 cipayung,/siung
2,toko dita kertosono,toko dita/
3,jl. orde baru,/jl. orde baru
4,raya samb gede 299 toko bb kids,toko bb kids/raya samb gede
...,...,...
268001,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya
268002,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani
268003,raya cila kko cilandak timur kel.,/raya cila kko
268004,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/


In [12]:
def getPOI(s) :
    return s.split("/")[0]

def getStreet(s) :
    return s.split("/")[1]

df["POI"] = df["POI/street"].apply(getPOI)
df["Street"] = df["POI/street"].apply(getStreet) 

In [13]:
def create_label(s, poi, street) :
    tokens = s.split(" ")
    n = len(tokens)
    tp = poi.split(" ")
    ts = street.split(" ")
    tags = []
    i = 0
    while i < n :
        if len(tp) > 0 and tokens[i] == tp[0] :
            j = 1
            while j < len(tp) :
                if i+j < n and tokens[i+j] == tp[j]:
                    j += 1
                else :
                    break
            if j == len(tp) :
                tags.append("B-ORGANISATION")
                for k in range(1, j) :
                    tags.append("I-ORGANISATION")
                i += j
            else :
                tags.append("O")
                i += 1
        elif len(ts) > 0 and tokens[i] == ts[0] :
            j = 1
            while j < len(ts) :
                if i+j < n and tokens[i+j] == ts[j] :
                    j += 1
                else :
                    break
            if j == len(ts) :
                tags.append("B-PLACE")
                for k in range(1, j) :
                    tags.append("I-PLACE")
                i += j
            else :
                tags.append("O")
                i += 1                
        else :
            tags.append("O")
            i += 1
        
#         print(tokens)
#         print(tags)

    if tokens[-1] != "." :
        tokens.append(".")
        tags.append("O")
    
    res = ""
#     print(tokens)
#     print(tags)
    for token, tag in zip(tokens, tags) :
        res += "{}\t{}\n".format(token, tag)
    return res

In [14]:
s = "raya samb gede , 299 toko bb kids"
poi = "toko bb kids"
street = "raya samb gede"
res = create_label(s, poi, street)
print(res)

raya	B-PLACE
samb	I-PLACE
gede	I-PLACE
,	O
299	O
toko	B-ORGANISATION
bb	I-ORGANISATION
kids	I-ORGANISATION
.	O



In [15]:
df["Label"] = df.apply(lambda x: create_label(x.raw_address, x.POI, x.Street), axis=1)

In [16]:
df

,raw_address,POI/street,POI,Street,Label
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika,jl\tB-PLACE\nkapuk\tI-PLACE\ntimur\tI-PLACE\nd...
1,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung,setu\tO\nsiung\tB-PLACE\n119\tO\nrt\tO\n5\tO\n...
2,toko dita kertosono,toko dita/,toko dita,,toko\tB-ORGANISATION\ndita\tI-ORGANISATION\nke...
3,jl. orde baru,/jl. orde baru,,jl. orde baru,jl.\tB-PLACE\norde\tI-PLACE\nbaru\tI-PLACE\n.\...
4,raya samb gede 299 toko bb kids,toko bb kids/raya samb gede,toko bb kids,raya samb gede,raya\tB-PLACE\nsamb\tI-PLACE\ngede\tI-PLACE\n2...
...,...,...,...,...,...
268001,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya,,kakap raya,karawaci\tO\nbaru\tO\nkakap\tB-PLACE\nraya\tI-...
268002,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,,jend ahmad yani,jend\tB-PLACE\nahmad\tI-PLACE\nyani\tI-PLACE\n...
268003,raya cila kko cilandak timur kel.,/raya cila kko,,raya cila kko,raya\tB-PLACE\ncila\tI-PLACE\nkko\tI-PLACE\nci...
268004,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,taman asri,,jalan\tO\ncipadu\tO\njaya\tO\ntaman\tB-ORGANIS...


In [17]:
final = df.sample(frac=1, random_state=0).reset_index(drop=True)
final

,raw_address,POI/street,POI,Street,Label
0,krendang kren tim gg. 4 52 a 8 tambora,/kren tim gg. 4,,kren tim gg. 4,krendang\tO\nkren\tB-PLACE\ntim\tI-PLACE\ngg.\...
1,dukuh jl.p jl.p p 32 rt 11 4 13550 kramat jati,/jl.p jl.p,,jl.p jl.p,dukuh\tO\njl.p\tB-PLACE\njl.p\tI-PLACE\np\tO\n...
2,negeri bumi putra pecel lele jakarta 34719,pecel lele jakarta/,pecel lele jakarta,,negeri\tO\nbumi\tO\nputra\tO\npecel\tB-ORGANIS...
3,toko t & d src grup kahen magelang selatan,toko t & d src grup/kahen,toko t & d src grup,kahen,toko\tB-ORGANISATION\nt\tI-ORGANISATION\n&\tI-...
4,scar barat i 26 pagedangan,/scar barat i,,scar barat i,scar\tB-PLACE\nbarat\tI-PLACE\ni\tI-PLACE\n26\...
...,...,...,...,...,...
268001,pt. sankei dharma indonesia permata raya sirna...,pt. sankei dharma indonesia/permata raya,pt. sankei dharma indonesia,permata raya,pt.\tB-ORGANISATION\nsankei\tI-ORGANISATION\nd...
268002,leuwigajah sad 6 cimahi selatan,/sad,,sad,leuwigajah\tO\nsad\tB-PLACE\n6\tO\ncimahi\tO\n...
268003,lin barat suma napa batang toru,/lin barat suma,,lin barat suma,lin\tB-PLACE\nbarat\tI-PLACE\nsuma\tI-PLACE\nn...
268004,baba toko azka sangkanhurip,toko azka/baba,toko azka,baba,baba\tB-PLACE\ntoko\tB-ORGANISATION\nazka\tI-O...


In [18]:
n = len(final)
nt = int(0.95 * n)
train = final[:nt+1]
test = final[nt:]

In [19]:
train

,raw_address,POI/street,POI,Street,Label
0,krendang kren tim gg. 4 52 a 8 tambora,/kren tim gg. 4,,kren tim gg. 4,krendang\tO\nkren\tB-PLACE\ntim\tI-PLACE\ngg.\...
1,dukuh jl.p jl.p p 32 rt 11 4 13550 kramat jati,/jl.p jl.p,,jl.p jl.p,dukuh\tO\njl.p\tB-PLACE\njl.p\tI-PLACE\np\tO\n...
2,negeri bumi putra pecel lele jakarta 34719,pecel lele jakarta/,pecel lele jakarta,,negeri\tO\nbumi\tO\nputra\tO\npecel\tB-ORGANIS...
3,toko t & d src grup kahen magelang selatan,toko t & d src grup/kahen,toko t & d src grup,kahen,toko\tB-ORGANISATION\nt\tI-ORGANISATION\n&\tI-...
4,scar barat i 26 pagedangan,/scar barat i,,scar barat i,scar\tB-PLACE\nbarat\tI-PLACE\ni\tI-PLACE\n26\...
...,...,...,...,...,...
254601,mentari motor rajma kal,mentari motor/rajma kal,mentari motor,rajma kal,mentari\tB-ORGANISATION\nmotor\tI-ORGANISATION...
254602,angkringan cokro h. o. s. cokroaminoto,angkringan cokro/h. o. s. cokroaminoto,angkringan cokro,h. o. s. cokroaminoto,angkringan\tB-ORGANISATION\ncokro\tI-ORGANISAT...
254603,jalan pasar kepuh blok 14 kuningan kuningan,/jalan pasar kepuh,,jalan pasar kepuh,jalan\tB-PLACE\npasar\tI-PLACE\nkepuh\tI-PLACE...
254604,tahir kranji bekasi barat,/tahir,,tahir,tahir\tB-PLACE\nkranji\tO\nbekasi\tO\nbarat\tO...


In [20]:
def save_file(filename, arr) :
    f = open(filename, "w+",  encoding='utf-8')
    for s in arr :
        f.write(s.encode("ascii", "ignore").decode())
        f.write("\n")
    f.close()

In [21]:
train_path = data_path + "bert-fine-tune/train.txt"
validation_path = data_path + "bert-fine-tune/validation.txt"

train_data = train["Label"].values.tolist()
test_data = test["Label"].values.tolist()


ntry = None
# ntry = 1000
if ntry :
    train_data = train_data[:ntry]
    test_data = test_data[:ntry]

save_file(train_path, train_data)
save_file(validation_path, test_data)